<a href="https://colab.research.google.com/github/devwatts/Arduino/blob/master/Electra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub


In [2]:
!pip install tensorflow_text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
df=pd.read_csv("amazon.csv")

In [4]:
df.head(5)

,S,Feedback,Sentiment
0,1,"Good case, Excellent value.",Positive
1,2,Great for the jawbone.,Positive
2,3,Tied to charger for conversations lasting more...,Negative
3,4,The mic is great.,Positive
4,5,I have to jiggle the plug to get it to line up...,Negative


In [5]:
df.groupby('Sentiment').describe()

S                                                           
           count        mean         std  min     25%    50%     75%    max
Sentiment                                                                  
Negative   499.0  515.356713  292.393469  3.0  259.50  528.0  768.50  999.0
Positive   500.0  484.674000  284.088374  1.0  246.75  476.5  732.25  978.0

In [6]:
df['Sentiment'].value_counts()

Positive    500
Negative    499
Name: Sentiment, dtype: int64

In [7]:
df = df.rename({'Sentiment': 'label'}, axis=1)

In [8]:
df['label'] = df['label'].replace(['Negative','Positive'],[0,1]) 

In [9]:
X = df['Feedback']
y = df['label']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [11]:
X_train.head(4)

615                                        Poor product.
949                                  It is a joy to use.
864                           Would recommend this item.
821    I have been very satisfied with this cell phon...
Name: Feedback, dtype: object

In [12]:
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
electra = hub.KerasLayer("https://tfhub.dev/google/electra_small/2")

In [38]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocess(text_input)
outputs = electra(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(16, activation='relu')(l)
l = tf.keras.layers.Dense(32, activation='relu')(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [39]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [40]:
len(X_train)

799

In [41]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [42]:
model.fit(X_train, y_train, epochs=30, batch_size= 32) 

Epoch 1/30
25/25 [==============================] - 6s 114ms/step - loss: 0.6797 - accuracy: 0.5670 - precision: 0.5714 - recall: 0.5743
Epoch 2/30
25/25 [==============================] - 3s 110ms/step - loss: 0.6455 - accuracy: 0.6508 - precision: 0.6640 - recall: 0.6262
Epoch 3/30
25/25 [==============================] - 3s 111ms/step - loss: 0.6177 - accuracy: 0.6621 - precision: 0.6650 - recall: 0.6683
Epoch 4/30
25/25 [==============================] - 3s 121ms/step - loss: 0.5868 - accuracy: 0.6783 - precision: 0.6746 - recall: 0.7030
Epoch 5/30
25/25 [==============================] - 3s 112ms/step - loss: 0.5614 - accuracy: 0.7259 - precision: 0.7295 - recall: 0.7277
Epoch 6/30
25/25 [==============================] - 3s 111ms/step - loss: 0.5534 - accuracy: 0.7284 - precision: 0.7275 - recall: 0.7401
Epoch 7/30
25/25 [==============================] - 3s 112ms/step - loss: 0.5435 - accuracy: 0.7309 - precision: 0.7405 - recall: 0.7203
Epoch 8/30
25/25 [=======================

In [43]:
model.evaluate(X_test, y_test)

7/7 [==============================] - 2s 96ms/step - loss: 0.4668 - accuracy: 0.7900 - precision: 0.8293 - recall: 0.7083


[0.46679002046585083,
 0.7900000214576721,
 0.8292682766914368,
 0.7083333134651184]

In [44]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

7/7 [==============================] - 1s 101ms/step


In [45]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0])

In [46]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

cm = confusion_matrix(y_test, y_predicted)
cm 
accuracy_score(y_test, y_predicted)

0.79

In [47]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.76      0.87      0.81       104
           1       0.83      0.71      0.76        96

    accuracy                           0.79       200
   macro avg       0.80      0.79      0.79       200
weighted avg       0.79      0.79      0.79       200

